TODO:

- [ ] replace id 'CIM00085577'

## Libraries

In [1]:
import sys
import os
import datetime as dt
import numpy as np
import pandas as pd
import geopandas as gpd
import altair as alt
from vega_datasets import data

In [2]:
if not 'mainDir' in globals():
    mainDir = os.path.dirname(os.getcwd()) # Get parent dir: os.path.dirname()
print(mainDir)

/Users/lassescheele/Documents/Private/altair-climate-change


## Settings

In [3]:
first_relevant_year = 1900

start_reference_period = 1961
end_reference_period = 1990

In [4]:
dir_input = os.path.join(mainDir,'data','raw')
dir_output = os.path.join(mainDir,'data','processed')

In [5]:
# ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/all/
list_station_ids = [
    'USW00025501', # Kodiak, Alaska, USA, 1945
    'CA002401200', # EUREKA, Canada, 1947
    'USW00023237', # Stockton, California, USA, 1941
    'USW00022521', # Honululu, Hawaii, USA, 1939
    #'FP000091929', # BORA-BORA/MOTU-MUTE, French Polynesia, 1943
    'USW00014939', # Lincoln Muni AP, Nebraska, USA, 1942
    'USW00094702', # Bridgeport, Conneticut, USA, 1942
    'COM00080028', # Ernesto Cortissoz, Colombia, 1941
    'BR00E4-1230', # Campinas, Brazil, 1951
    'CIM00085577', # Quinta Normal (Santiago de Chile), Chile, 1946
    'GLM00004390', # IKERASASSUAQ, Greenland, 1943
    'GM000010147', # Hamburg, Germany, 1939
    'RSM00027612', # Moscow, Russia, 1936
    'KZ000035188', # Astana, Kazakhstan, 1881
    'RSM00021921', # KJUSJUR, Russia, 1909
    'RSM00032618', # NIKOLSKOYE/BERINGA OSTROV, Russia, 1899
    'TSM00060715', # CARTHAGE, Tunisia, 1943
    'SG000061641', # Dakar, Senegal, 1943
    'SU000062770', # Genina, Sudan, 1943
    'MUM00041316', # SALALAH, Oman, 1943
    'SF001344780', # Calvinia, South Africa, 1957
    'SE000063980', # SEYCHELLES INTERNAT, Seychelles, 1957
    'IN022021900', # New Delhi, India, 1942
    'CHM00054511', # Beijing, China, 1945
    'JA000047759', # Kyoto, Japan, 1945
    'RP000098429', # Manila, Philippines, 1945
    'FJ000091680', # NADI AIRPORT, Fiji, 1942
    'ASN00003003', # BROOME AIRPORT, Australia, 1939
    'ASN00066037', # Sydney Airport, Australia, 1943
    'AYM00089022', # Halley, Antarctica, 1956
    'SXM00088903', # GRYTVIKEN, South Georgia and the South Sandwich Islands [United Kingdom], 1953
    'AYM00089542', # MOLODEZNAJA, Antarctica, 1973
    'AYM00089606', # Vostok, Antarctica, 1973
]
#list_station_ids

## Read data

### Read stations

In [37]:
df_stations = pd.read_csv(os.path.join(dir_input,"ghcn","weather-stations-tavg.csv"))
df_stations.shape

(9447, 12)

In [38]:
df_stations = df_stations.loc[df_stations['id'].isin(list_station_ids)]
df_stations.shape

(32, 12)

In [39]:
df_stations['name'] = df_stations['name'].str.title()
for column in ['name','country_code','country_name','state']:
    df_stations[column] = df_stations[column].str.strip()
#df_stations['wmoid'] = df_stations['wmoid'].astype("Int64")
df_stations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32 entries, 72 to 9184
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            32 non-null     object 
 1   element       32 non-null     object 
 2   firstyear     32 non-null     int64  
 3   lastyear      32 non-null     int64  
 4   wmoid         32 non-null     float64
 5   latitude      32 non-null     float64
 6   longitude     32 non-null     float64
 7   elevation     32 non-null     float64
 8   name          32 non-null     object 
 9   country_code  32 non-null     object 
 10  state         32 non-null     object 
 11  country_name  32 non-null     object 
dtypes: float64(4), int64(2), object(6)
memory usage: 3.2+ KB


In [40]:
df_stations

,id,element,firstyear,lastyear,wmoid,latitude,longitude,elevation,name,country_code,state,country_name
72,FJ000091680,TAVG,1942,2020,91680.0,-17.7500,177.4500,18.0,Nadi Airport,FJ,,Fiji
468,MUM00041316,TAVG,1943,2020,41316.0,17.0390,54.0910,22.3,Salalah,MU,,Oman
484,COM00080028,TAVG,1941,2020,80028.0,10.8900,-74.7810,29.9,Ernesto Cortissoz,CO,,Colombia
522,SXM00088903,TAVG,1953,2020,88903.0,-54.2830,-36.5000,3.0,Grytviken,SX,,South Georgia and the South Sandwich Islands [...
586,SE000063980,TAVG,1957,2020,63980.0,-4.6670,55.5170,3.0,Seychelles Internat,SE,,Seychelles
601,RP000098429,TAVG,1945,2020,98429.0,14.5170,121.0000,15.0,Ninoy Aquino Intern,RP,,Philippines
849,CIM00085577,TAVG,1946,2020,85577.0,-33.4330,-70.6830,520.0,Quinta Normal,CI,,Chile
1056,SG000061641,TAVG,1943,2020,61641.0,14.7330,-17.5000,24.0,Dakar/Yoff,SG,,Senegal
1063,SU000062770,TAVG,1943,2014,62770.0,13.4830,22.4500,805.0,Genina,SU,,Sudan
1116,TSM00060715,TAVG,1943,2020,60715.0,36.8510,10.2270,6.7,Carthage,TS,,Tunisia


### Read climate data

In [10]:
df = pd.DataFrame()
for index, row in df_stations[:].iterrows():
    file_path = os.path.join(dir_input,"ghcn",f"{row['id']}.dly")
    print(row['id'], file_path)
    df_temp = pd.read_csv(file_path, header=None)
    #print(df_temp.shape)
    #print(df_temp.head())
    df_temp['index'] = df_temp[0].str[:21]
    df_temp['id'] = df_temp['index'].str[:11]
    df_temp['year'] = df_temp['index'].str[11:15].astype('int64')
    df_temp['month'] = df_temp['index'].str[15:17].astype('int64')
    df_temp['dt'] = pd.to_datetime(df_temp['year']*10000+df_temp['month']*100+1,format='%Y%m%d')
    df_temp['element'] = df_temp['index'].str[17:]
    df_temp = df_temp.loc[(df_temp['year']>=first_relevant_year) & (df_temp['element']=='TAVG')]
    #print(df_temp.shape)
    #print(df_temp.head())
    
    #df_temp['values'] = df_temp[0].str[21:].str.replace('-9999','-9999 -9999')
    df_temp_values = df_temp[0].str[21:].str.replace('-9999',' -9999 XX').str.replace('HO',' ').str.replace('H',' ').str.strip().str.split("\s+", n = None, expand = True)
    df_temp_values = (df_temp_values[[x for x in df_temp_values.columns if not x%2]].astype('float').replace(-9999.0,pd.NA) / 10).mean(axis=1)
    #df_temp_values.columns = range(1,(len(df_temp_values.columns)+1))
    #df_temp_values['index'] = df_temp['index']
    #print(df_temp_values.shape)
    #print(df_temp_values.head())
    
    df_temp['value'] = df_temp_values
    df_temp = df_temp.drop([0,'index'], axis=1)
    #df_temp = pd.merge(df_temp,df_temp_values,on='index', how='left')#.rename(columns={'AverageTemperature_ref':'AverageTemperature_sinceRef'})
    #print(df_temp.shape)
    #print(df_temp.head())
    
    if len(df) == 0:
        df = df_temp.copy()
    else:
        df = df.append(df_temp)
print(df.shape)
print(df.head())

FJ000091680 /Users/lassescheele/Documents/Private/altair-climate-change/data/raw/ghcn/FJ000091680.dly
MUM00041316 /Users/lassescheele/Documents/Private/altair-climate-change/data/raw/ghcn/MUM00041316.dly
COM00080028 /Users/lassescheele/Documents/Private/altair-climate-change/data/raw/ghcn/COM00080028.dly
SXM00088903 /Users/lassescheele/Documents/Private/altair-climate-change/data/raw/ghcn/SXM00088903.dly
SE000063980 /Users/lassescheele/Documents/Private/altair-climate-change/data/raw/ghcn/SE000063980.dly
RP000098429 /Users/lassescheele/Documents/Private/altair-climate-change/data/raw/ghcn/RP000098429.dly
CIM00085577 /Users/lassescheele/Documents/Private/altair-climate-change/data/raw/ghcn/CIM00085577.dly
SG000061641 /Users/lassescheele/Documents/Private/altair-climate-change/data/raw/ghcn/SG000061641.dly
SU000062770 /Users/lassescheele/Documents/Private/altair-climate-change/data/raw/ghcn/SU000062770.dly
TSM00060715 /Users/lassescheele/Documents/Private/altair-climate-change/data/raw/g

## Prepare data

### Heatmap data

In [11]:
df_reference = df.loc[(df['year']>=start_reference_period) & (df['year']<=end_reference_period)]
df_reference = pd.DataFrame(df_reference.groupby(['id','month'])['value'].mean())
df_reference

value
id          month           
ASN00003003 1      29.551387
            2      29.269170
            3      29.369731
            4      28.117939
            5      24.845111
...                      ...
USW00014939 8      24.284946
            9      18.311111
            10     14.944086
            11      5.115556
            12     -4.744086

[324 rows x 1 columns]

In [12]:
df.loc[(df['id']=='CIM00085577'),'year'].unique()

array([1946, 1947, 1948, 1949, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
       2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])

In [14]:
list_ids = df['id'].unique().tolist()
counter = 1
df['value_ref'] = np.nan
for id in list_ids:
    if counter % 10 == 0:
        print(f"{counter} of {len(list_ids)} stations")
    for month in range(1,13):
        try:
            ref_value = df_reference.loc[(id,month),'value'].item()
        except:
            ref_value = np.nan
        value = df.loc[(df['id']==id) & (df['month']==month),'value']#.item()
        #print(month,value,ref_value)
        if not np.isnan(ref_value):
            df.loc[(df['id']==id) & (df['month']==month),'value_ref'] = value - ref_value
    counter += 1

10 of 32 stations
20 of 32 stations
30 of 32 stations


In [15]:
df = df.sort_values(['id','year','month'], ascending=[True,True,True])

## Plot point map

In [16]:
first_analysis_year = 1950
last_analysis_year = 2020

In [17]:
#alt.data_transformers.enable('data_server') # data will be served in the background rather than embedded in the chart specification
#alt.data_transformers.enable('json')        # data will be serialized to disk and referenced by URL
alt.data_transformers.enable('default')    # data will be fully embedded in the notebook

DataTransformerRegistry.enable('default')

In [18]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [19]:
#df['AverageTemperature_ref'].quantile([0.05,.25,.5,.75,.95]).values.tolist()

In [20]:
domain = [-5,-.5,0,.5,5]
range_ = ['#0571b0','#92c5de','#f7f7f7','#f4a582','#ca0020']

In [42]:
sphere = alt.sphere()
graticule = alt.graticule()

source_map = alt.topo_feature(data.world_110m.url, 'countries')
source_heatmap = df.loc[df['year'].isin(list(range(first_analysis_year,last_analysis_year+1)))]
source_trend = source_heatmap.groupby(['id','year'])['value_ref'].mean().reset_index()

width = 900*1.25
height = 450*1.25

default_station = "GM000010147"

selector = alt.selection(
    type="single", fields=['id'], init={"id": default_station}
)

color = alt.condition(selector,alt.value('#FF00E8'),alt.value('#2ECC71'))
size = alt.condition(selector,alt.value(400),alt.value(200))
strokeWidth = alt.condition(selector,alt.value(4),alt.value(3))

background = alt.layer(
    alt.Chart(sphere).mark_geoshape(fill='#D5F5FF'),
    alt.Chart(graticule).mark_geoshape(stroke='white', strokeWidth=0.5),
    alt.Chart(source_map).mark_geoshape(fill='lightgrey', stroke='grey', strokeWidth=0.5)
).project(
    'naturalEarth1'
).properties(
    width=width, height=height
)
'''
choropleth = alt.Chart(
    source_map, title=f'Average monthly temperature change for {end_reference_period+1}-{last_year} compared to {start_reference_period}-{end_reference_period}'
).mark_geoshape(
    stroke='grey', strokeWidth=0.5
).encode(
    color=color,
    tooltip=[
        'Country:N',
        alt.Tooltip('AverageTemperature_sinceRef:Q', title=f'Average monthly temperature change for {end_reference_period+1}-{last_year} compared to {start_reference_period}-{end_reference_period} (°C)', format='+.2f'),
    ]
).project(
    'naturalEarth1'
).properties(
    width=width, height=height
).add_selection(
    selector
)
'''
points = alt.Chart(
    df_stations
).mark_circle(
    stroke='black',
).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color=color,
    size=size,
    strokeWidth=strokeWidth,
    tooltip=['name:N','country_name:N', 'firstyear:O', 'lastyear:O']
).add_selection(
    selector
)

trend = alt.Chart(
    source_trend#, title=f'Average monthly temperature {first_analysis_year}-{last_analysis_year} (compared to the period {start_reference_period}-{end_reference_period})'
).mark_line().encode(
    x=alt.X('year:O', sort=alt.EncodingSortField('year', order='ascending')),
    y=alt.Y('value_ref:Q'),
).add_selection(
    selector
).transform_filter(
    selector
).properties(
    #width=height/((last_analysis_year-first_analysis_year)/12),
    #height=height
    width=width,
    height=width/((last_analysis_year-first_analysis_year)/12),
)

heatmap = alt.Chart(
    source_heatmap, title=f'Average monthly temperature {first_analysis_year}-{last_analysis_year} (compared to the period {start_reference_period}-{end_reference_period})'
).mark_rect(
    #stroke='grey',
    #strokeWidth=0.5,
).encode(
    y=alt.Y('month:O', sort=alt.EncodingSortField('month', order='ascending')),
    x=alt.X('year:O', sort=alt.EncodingSortField('year', order='ascending')),
    color=alt.Color(
        'value_ref:Q',
        scale=alt.Scale(domain=domain, range=range_),
        #scale=alt.Scale(type='sqrt', scheme="redyellowblue", order="descending"), # domain=[max_value, -max_value]
        title='Temperature difference (°C)'
    ),
    tooltip=[
        'id:N','year:O','month:O',
        alt.Tooltip('value_ref:Q', title=f'Average monthly temperature change compared to {start_reference_period}-{end_reference_period} (°C)', format='+.2f'),
    ]
).add_selection(
    selector
).transform_filter(
    selector
).properties(
    #width=height/((last_analysis_year-first_analysis_year)/12),
    #height=height
    width=width,
    height=width/((last_analysis_year-first_analysis_year)/12),
)

#chart = points # ((background + points) & heatmap).configure_legend(orient='right').configure_view(stroke=None)
chart = alt.vconcat(heatmap, trend, alt.layer(background, points))#.configure_legend(orient='right').configure_view(stroke=None)
chart

alt.VConcatChart(...)

In [ ]:
file_name = f"weather_stations_tavg_{first_analysis_year}-{last_analysis_year}_ref{start_reference_period}-{end_reference_period}_map_heatmap"
print(file_name)

In [ ]:
chart.save(os.path.join(mainDir,'docs',file_name+'.html'))
#chart.save(os.path.join(mainDir,'plots',file_name+'.png'), scale_factor=1.5)